**Otto Recommender System:** Converting train.jsonl file to TFRecord file format. The idea is to use TFRecord file format further and learn TensorFlow Recommenders (TFRS) while implementing this usecase  

Following resources gave me good understanding of WHAT, WHY and How of TFRecord file format. 

* https://www.tensorflow.org/tutorials/load_data/tfrecord
* https://keras.io/examples/keras_recipes/creating_tfrecords/
* https://www.kaggle.com/code/ryanholbrook/tfrecords-basics/notebook

As I start to learn TensorFlow, your remarks / comments for sure can help in improving my knowledge 

In [1]:
import numpy as np 
import pandas as pd 
import os
import zipfile
import json
from tqdm import tqdm
import tensorflow as tf

In [2]:
INPUT_DIR = '../input/otto-recommender-system'
TRAIN_FILE = f'{INPUT_DIR}/train.jsonl'
TEST_FILE = f'{INPUT_DIR}/test.jsonl'

In [3]:
def _int64_feature(value):
    # Returns an int64_list from a bool / enum / int / uint
    return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def serialize_example(session, aid, ts, typ):
    # Creates a tf.train.Example message ready to be written to a file.
    # Create a dictionary mapping the feature name to the tf.train.Example-compatible data type.
    feature = {
          'session': _int64_feature(session),
          'aid': _int64_feature(aid),
          'ts': _int64_feature(ts),
          'typ': _int64_feature(typ),
      }
    # Create a Features message using tf.train.Example.
    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    return example_proto.SerializeToString()

In [4]:
def features_list(train_df):
    session = []
    aid = []
    ts = []
    typ = []
    codes = {'clicks':1, 'carts':2, 'orders':3}

    for index in train_df.index:
        session_id = train_df['session'][index]
        for event in train_df['events'][index]:
            session.append(session_id)
            aid.append(event['aid'])
            ts.append(event['ts'])
            typ.append(codes[event['type']])
    return session, aid, ts, typ

def write_tfrecord(file_num, session, aid, ts, typ):
    filename = f'train_chunk_{file_num}.tfrecord'
    options = tf.io.TFRecordOptions(compression_type='ZLIB')
    
    with tf.io.TFRecordWriter(filename, options=options) as writer:
        for i in range(len(session)):
            example = serialize_example(session[i], aid[i], ts[i], typ[i])
            writer.write(example)
    zip_tfrecord(filename)
    return

def zip_tfrecord(filename):
    OUTPUT_DIR = '/kaggle/working/'
    zip_file = f'{OUTPUT_DIR}/otto_tfrecord.zip'
    filename = f'{OUTPUT_DIR}/{filename}'
    
    with zipfile.ZipFile(zip_file, 'a') as zf:
        zf.write(filename)
    os.remove(filename)
    return

In [5]:
%%time
num_lines = sum(1 for line in open(TRAIN_FILE))
print('Number of lines: ', num_lines)

chunk_size=100000
num_chunks= int(np.ceil(num_lines / chunk_size))
print('Number of chunks: ',num_chunks)

Number of lines:  12899779
Number of chunks:  129
CPU times: user 10 s, sys: 5.81 s, total: 15.8 s
Wall time: 2min 11s


In [6]:
%%time
chunks = pd.read_json(TRAIN_FILE, lines=True, chunksize=chunk_size)

for e, chunk in tqdm(enumerate(chunks)):
    print('Processing chunk # : ', e)
    session, aid, ts, typ = features_list(chunk)
    write_tfrecord(e, session, aid, ts, typ)

0it [00:00, ?it/s]

Processing chunk # :  0


1it [04:03, 243.77s/it]

Processing chunk # :  1


2it [07:59, 239.05s/it]

...

Processing chunk # :  127


128it [2:53:44, 41.62s/it]

Processing chunk # :  128


129it [2:54:20, 81.09s/it]

CPU times: user 2h 51min 21s, sys: 1min 17s, total: 2h 52min 38s
Wall time: 2h 54min 20s
